<a href="https://colab.research.google.com/github/hopperrr/Weather_radar/blob/main/Radar_to_vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://mrms.ncep.noaa.gov/data/2D/CREF_1HR_MAX/

Grab data and convert to vector

In [ ]:
#!pip install leafmap
!pip install geopandas
#!pip install topojson
!pip install rasterio

In [ ]:
!wget https://mrms.ncep.noaa.gov/data/2D/CREF_1HR_MAX/MRMS_CREF_1HR_MAX.latest.grib2.gz
!gzip -d /content/MRMS_CREF_1HR_MAX.latest.grib2.gz

--2022-11-01 23:02:30--  https://mrms.ncep.noaa.gov/data/2D/CREF_1HR_MAX/MRMS_CREF_1HR_MAX.latest.grib2.gz
Resolving mrms.ncep.noaa.gov (mrms.ncep.noaa.gov)... 140.90.101.53
Connecting to mrms.ncep.noaa.gov (mrms.ncep.noaa.gov)|140.90.101.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2456437 (2.3M) [application/x-gzip]
Saving to: ‘MRMS_CREF_1HR_MAX.latest.grib2.gz’

MRMS_CREF_1HR_MAX.l 100%[===================>]   2.34M  1.12MB/s    in 2.1s    

2022-11-01 23:02:33 (1.12 MB/s) - ‘MRMS_CREF_1HR_MAX.latest.grib2.gz’ saved [2456437/2456437]



In [ ]:
#https://gis.stackexchange.com/questions/37790/reprojecting-raster-from-0-360-to-180-180-with-cutting-180-meridian-using-gdalw
!gdalwarp -s_srs "+proj=longlat +ellps=WGS84" -t_srs WGS84 MRMS_CREF_1HR_MAX.latest.grib2 180.tif  -wo SOURCE_EXTRA=1000 --config CENTER_LONG 0
#!gdal_translate -a_nodata -99 -ot Float32 -projwin -74.6793 43.0922 -68.3148 40.2055 180.tif small_area.tif
!gdal_translate -a_nodata -99 -ot Float32 -projwin -103.025 36.829 -84.708 25.080 180.tif small_area.tif
#Switch order of lat and long
#Texas 36.829,-103.025  25.080,-84.708
#43.0922,-74.6793 40.2055,-68.3148

Creating output file that is 7000P x 3500L.
Processing input file MRMS_CREF_1HR_MAX.latest.grib2.
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7000, 3500
0...10...20...30...40...50...60...70...80...90...100 - done.


In [ ]:
#9/22/2022 Use this for exporting
#Much faster than polygonize!
import gdal
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import os
from google.colab import files

import rasterio
from rasterio.features import shapes
mask = None
results=None
with rasterio.Env():
    with rasterio.open('/content/small_area.tif') as src:
        image = src.read(1) # first band
        results = (
     
        {'properties': {'DN': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=mask, transform=src.transform)))
        
geoms=list(results)        

import geopandas as gp

gdf = geopandas.GeoDataFrame.from_features(geoms) #Convert to Geopandas dataframe

gdf.insert(0, 'New_ID', range(0, 0 + len(gdf)))

gdf.head()

gdf.set_crs("EPSG:4326", inplace=True)
gdf_good = gdf[gdf['DN'] > -90]
gdf_good.to_file("radar.gpkg", driver='GPKG')
files.download('/content/radar.gpkg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>